In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

dataset,info = tfds.load("tf_flowers",as_supervised=True,with_info=True)
dataset_size = info.splits["train"].num_examples#3670
class_names = info.features["label"].names
n_classes = info.features["label"].num_classes#5

# Define the input shape
input_shape = (224, 224, 3)

# Normalize the images and resize them
def preprocess(image, label):
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize_with_pad(image, 224, 224)
    return image, label

# Load the data
data = tfds.load("tf_flowers", split="train[:90%]", as_supervised=True)
test_data = tfds.load("tf_flowers", split="train[90%:]", as_supervised=True)
train_data = data.take(2090)
valid_data = data.skip(2090)

train_data = train_data.map(preprocess).batch(32)
valid_data = valid_data.map(preprocess).batch(32)
test_data = test_data.map(preprocess).batch(32)

# Define the model
model = tf.keras.Sequential([
    # Encoder
    tf.keras.layers.Conv2D(64, 3, activation='relu', input_shape=input_shape, padding='same'),
    tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same'),
    tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
    tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
    tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(),
    # Bottleneck
    tf.keras.layers.Conv2D(512, 3, activation='relu', padding='same'),
    tf.keras.layers.Conv2D(512, 3, activation='relu', padding='same'),
    tf.keras.layers.Conv2D(512, 3, activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(),
    # Decoder
    tf.keras.layers.Conv2DTranspose(256, 3, activation='relu', padding='same'),
    tf.keras.layers.Conv2DTranspose(256, 3, activation='relu', padding='same'),
    tf.keras.layers.Conv2DTranspose(256, 3, activation='relu', padding='same'),
    tf.keras.layers.UpSampling2D(),
    tf.keras.layers.Conv2DTranspose(128, 3, activation='relu', padding='same'),
    tf.keras.layers.Conv2DTranspose(128, 3, activation='relu', padding='same'),
    tf.keras.layers.UpSampling2D(),
    tf.keras.layers.Conv2DTranspose(64, 3, activation='relu', padding='same'),
    tf.keras.layers.Conv2DTranspose(64, 3, activation='relu', padding='same'),
    tf.keras.layers.UpSampling2D(),
    tf.keras.layers.Conv2D(n_classes, 1, activation='softmax')
])


# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])



In [7]:
import tensorflow as tf
import tensorflow_datasets as tfds

# Load the dataset
dataset, info = tfds.load('tf_flowers', with_info=True, as_supervised=True)

# Load the data
data = tfds.load("tf_flowers", split="train[:90%]", as_supervised=True)
test_data = tfds.load("tf_flowers", split="train[90%:]", as_supervised=True)
train_data = data.take(2090)
valid_data = data.skip(2090)

def preprocess(image, label):
    # resize and normalize the image
    image = tf.image.resize(image, (224, 224))
    image = tf.cast(image, tf.float32) / 255.0

    # Remove the dimension of size 1
    image = tf.squeeze(image)

    # Return the result
    return image, label

# Prepare the data
train_data = data.take(2090)
valid_data = data.skip(2090)

train_data = train_data.map(preprocess)
train_data = train_data.batch(32)

valid_data = valid_data.map(preprocess)
valid_data = valid_data.batch(32)



base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(None, None, 3))
base_model.trainable = False

# Add new layers
input_tensor = base_model.input
x = base_model.output
x = tf.keras.layers.Conv2D(5, (1, 1), activation='softmax')(x)
model = tf.keras.Model(inputs=input_tensor, outputs=x)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


# Train the model
history = model.fit(train_data, epochs=10, validation_data=valid_data)


Epoch 1/10


ValueError: ignored